In [1]:
import pandas as pd
import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
import base64
import requests
import bs4
from bs4 import BeautifulSoup
import re
#import lxml.etree

#pip install geopandas
#pip install spacy
#python -m spacy download en_core_web_sm
#pip install lxml
#pip install parser-libraries 


In [2]:
"""Decode encoded Google News entry URLs."""
import base64
import functools
import re

# Ref: https://stackoverflow.com/a/59023463/

_ENCODED_URL_PREFIX = "https://news.google.com/articles/"
_ENCODED_URL_RE = re.compile(fr"^{re.escape(_ENCODED_URL_PREFIX)}(?P<encoded_url>[^?]+)")
_DECODED_URL_RE = re.compile(rb'^\x08\x13".+?(?P<primary_url>http[^\xd2]+)\xd2\x01')


@functools.lru_cache(2048)
def _decode_google_news_url(url: str) -> str:
    match = _ENCODED_URL_RE.match(url)
    encoded_text = match.groupdict()["encoded_url"]  # type: ignore
    encoded_text += "==="  # Fix incorrect padding. Ref: https://stackoverflow.com/a/49459036/
    decoded_text = base64.urlsafe_b64decode(encoded_text)

    match = _DECODED_URL_RE.match(decoded_text)
    primary_url = match.groupdict()["primary_url"]  # type: ignore
    primary_url = primary_url.decode()
    return primary_url


def decode_google_news_url(url: str) -> str:  # Not cached because not all Google News URLs are encoded.
    """Return Google News entry URLs after decoding their encoding as applicable."""
    return _decode_google_news_url(url) if url.startswith(_ENCODED_URL_PREFIX) else url

In [40]:
rz = pd.read_csv("full_rizz.csv", sep = ";")

In [41]:
# Create an empty list for the urls
urls = []

# loop through the urls in the column that contains links to googlenews articles
for url in rz['link']:
    try:
        new = decode_google_news_url(url)
        urls.append(new)
    
    # Attribute error caused by a faulty link will be removed from dataset
    except AttributeError:
        rz.drop(rz.loc[rz.link == url].index, inplace = True)

# After dropping rows we will need to reset the index
rz = rz.reset_index(drop=True)

In [43]:
# The lengths of the list and dataframe will be the same, as will their order
# We can concatenate into a new dataframe
rz = pd.concat([rz, pd.DataFrame({'url':urls})], axis=1)
# Store the dataframe 
rz.to_csv("updated_urls_rizz.csv", sep = ';', index = False)

In [62]:
from requests.structures import CaseInsensitiveDict
import requests

sharecount = []
for url in urls:
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    token = "EAAIxJUBe0zYBAOKUVYhZCHWRlZBm0Pj4M8DGka4LvaFuyeKCMLF8NG2rGtamLjwAMZBSrMFWPY6ZAnpZAihDhP7wjs9drlJ4W8C1ZCCyguI2Kg8rxgfUVrv22sFrcNLXYS1PYZAZALqr2ZB4bLpKMYHC7yGA7lXF8tf3G6BOI6YdQArlLt5bzY6TMZC73HTBr4ztpbRkSefUXGFZB9TTgbyZBL8nY8sSq5YuZAFy9owthwenPOMvK3K9ZACtP5hBiY2GfV4GcZD"
    resp = requests.get(f'https://graph.facebook.com/?id={url}&fields=engagement&access_token={token}', headers=headers).json()
    #print(resp)
    
    #break
    try:
        fb_shares = (resp['engagement']['share_count'])
        sharecount.append(fb_shares)
    except KeyError:
        print("API Limit exceeded")
        
        break
    


API Limit exceeded


In [65]:
len(sharecount)

0

In [39]:
from requests.structures import CaseInsensitiveDict

sharecount = []
for url in urls: 
    
    api = "https://api.sharedcount.com/v1.1?apikey=2a0336186dc8f5740fef9c1eb81947d66119c29f&url=" + url
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    resp = requests.get(api, headers=headers)
    details = resp.json()
    #for n in range(len(details)):
        #count = details[n]['Facebook']['share_count']
        #print(count)
        
    print(details)
    break
        #break
   # try:
     #   fb_count = details['Facebook']['share_count']
     #   sharecount.append(fb_count)
        
   # except KeyError:
       # print(url, details)
        #rz.drop(rz.loc[rz.url == url].index, inplace = True)
    


ConnectionError: HTTPSConnectionPool(host='count.donreach.com', port=443): Max retries exceeded with url: /?url=https://en.tempo.co/read/806444/sumatran-rhino-population-critical/&providers=all (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc7509d86d0>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))

In [63]:
rz['url']

0       https://en.tempo.co/read/806444/sumatran-rhino...
1       https://en.tempo.co/read/878302/javan-rhinos-t...
2       https://www.horsetalk.co.nz/2014/06/05/califor...
3       https://www.theguardian.com/science/2003/jul/0...
4       http://news.bbc.co.uk/2/hi/science/nature/3557...
                              ...                        
2188    https://www.michigansthumb.com/news/article/Vo...
2189    https://www.escondidograpevine.com/2020/05/30/...
2190    https://www.scubadivermag.com/fin-whale-washed...
2191    https://www.deccanherald.com/national/east-and...
2192    https://lonzowire.usatoday.com/2020/05/30/lonz...
Name: url, Length: 2193, dtype: object